# Tutorial: Creating a High-Resolution Custom ECOCLIMAP Database from Geoclimate for Meso-NH/Surfex.

## Introduction

Surface discretization in Meso-NH is already possible through ECOCLIMAP, but the resolution is 1km. For realistic simulations requiring a finer mesh, higher resolution surface discretisation becomes necessary. This tutorial provides an example of creating a high-resolution custom ECOCLIMAP database for Meso-NH/Surfex. In the final step of the tutorial, we also show how to achieve the same with topography from any given database.
To obtain a high-resolution cartography of surface types, we will use OpenStreetMap data, retrieved using the GeoClimate code. Post-processing steps will be required to adapt the GeoClimate output files for use as input in Meso-NH/Surfex, using the ECOCLIMAP cover system.



## Step 0: Retrieve OpenStreetMap Data with Geoclimate

The first step is to download Geoclimate: 
https://github.com/orbisgis/geoclimate/wiki/Download

You can then follow the tutorial below to execute your first case within your area of interest:
https://github.com/orbisgis/geoclimate/wiki/Default-case-with-OSM-for-Linux

>Note: If you wish to define an area using coordinates, you can do so in the configuration file as follows:

```{
    "description": "Processing OSM data",
    "input": {
        "locations": [
            [38.6,-75.4,38.99,-74.9]
          
        ],
        "area": 2000
    },
 ```   

>Note: To easily find the coordinates of a domaine, you can use http://bboxfinder.com/ and the "Draw rectangle" tool; be sure to select "Lat / Lng" in the bottom right. Note that by default, the surface area of the zone is limited to 1000km². If you want a larger area, you need to specify : `"area": XXX`.

>Note: Please make sure that the selected domain is larger than your domain in Meso-NH !

You can then visualize the Geoclimate output files in .geojson format in software like QGIS for example. For our case, where we are interested in surface types, only the files urban_areas.geojson, vegetation.geojson, and water.geojson are relevant.

To visualize the different surface TYPES in each .geojson file in QGIS, right-click on the file at the bottom left, then go to Properties/Symbology. Select "Categorized" at the top of the window, then choose TYPE under Value. Click on Classify at the bottom left and finally click OK.

You will observe that each surface unit, also called "polygons," has a surface TYPE (e.g., wood, sea, residential, grass, etc.).

If you do this for the three files of interest (`urban_areas.geojson`, `vegetation.geojson`, and `water.geojson`) based on the location of your domain, you may notice that the definition of the surface TYPE is not continuous; there are white zones.

<img src="Illustration/Step0.png" alt="Mon Image Locale" width="700" />

## Step 1: Fill in the Empty Zones

To manually fill in these areas, we will repurpose the `rsu_utrf_floor_area.geojson` file, where all polygons are defined. To keep only the polygons not defined in `urban_areas.geojson`, `vegetation.geojson`, and `water.geojson`, we will subtract their polygons from those of `rsu_utrf_floor_area.geojson`.

To achieve this, we will use the "difference" function defined in QGIS. A Python script that successively executes this function on the three files is available and named `Step_1_filter_QGIS.py`. Simply modify the path and specify the folder containing the .geojson files generated by Geoclimate, then execute the script.

The result of this script is a `unknown_areas.geojson` file, grouping all the polygons where GeoClimate did not find a surface TYPE. If you visualize `urban_areas.geojson`, `vegetation.geojson`, `water.geojson`, and `unknown_areas.geojson` in QGIS, you will now have a continuous set of polygons or surface definition.

<img src="Illustration/Step1.png" alt="Mon Image Locale" width="700" />


## Step 2: Associate an ECOCLIMAP cover with each surface type

Now, it is necessary to assign an ECOCLIMAP cover to each surface TYPE. To automate this, the Python script `Step_2_create_csv.py` reads each .geojson file and creates a corresponding csv file containing the list of present TYPEs. The user must then manually fill the N_COVER and Description columns by choosing the TYPE→ N_COVER association from the list available here: https://www.umr-cnrm.fr/surfex/spip.php?article219

<img src="Illustration/Tab1.1.png" alt="Mon Image Locale" width="400" />
<img src="Illustration/Tab1.2.png" alt="Mon Image Locale" width="400" />

>Note: Assigning the cover for the unknown_areas.geojson file is done in the next step.

## Step 3: Modify the geojson files

We will now use these csv files to add the N_COVER property to each polygon in every .geojson file. For the unknown_areas.geojson file, we assign a single cover without distinction. This choice is made directly by changing the `unknown_COVER` variable in the Python script `Step_3_add_N_COVER_.py`.

By executing this script, all four .geojson files now have a new N_COVER property for each of their polygons.

To visualize this, you can proceed in the same way as before by right-clicking on the file at the bottom left, then go to Properties/Symbology. Select "Categorized" at the top of the window, then choose N_COVER under Value. Click on Classify at the bottom left and finally click OK.

<img src="Illustration/Step2.png" alt="Mon Image Locale" width="700" />

>Note: This script also creates a new csv file from the existing ones by removing duplicates and assigning colors randomly to each COVER. This file will be used for the legend and mapping the covers (step 9).

<img src="Illustration/Tab3.1.png" alt="Mon Image Locale" width="400" />

## Step 4: Merge the geojson files

Now that we have a complete map of our domain with ECOCLIMAP cover numbers, the goal is to convert this into a format readable by Meso-NH/Surfex.

To start, we will merge our 4 .geojson files into one by concatenating them using the `Step_4_concatene.py` script.

<img src="Illustration/Step4.png" alt="Mon Image Locale" width="700" />

## Step 5: Rasterize the surface map

We rasterize the .geojson file from step 4 to convert its vector definition of surfaces into a discrete definition (pixels). For this, we use the QGIS "rasterize" function, which requires specifying the desired pixel size in meters as input. In the `Step_5_raster_QGIS.py` script, this is done with the variables `width_pixel` and `height_pixel`. The output of this function is a .tif file, which is essentially a pixelated image where the color intensity corresponds to the cover number.

<img src="Illustration/Step5.png" alt="Mon Image Locale" width="700" />

## Step 6: Writing the map to an ASCII file and input into Meso-NH/Surfex

To make this image readable by Meso-NH/Surfex, it needs to be converted into a csv file with 3 columns: `['Latitude [deg]', 'Longitude [deg]', 'N_COVER'[-]`. In .geojson and .tif files, the position is not in Latitude Longitude but in Universal Transverse Mercator (UTM).
UTM divides the Earth into 60 zones and projects each into a plane as a basis for its coordinates. So, to convert to Latitude Longitude, the Python script needs to know the UTM zone and the geodetic system in use. 

`utm_proj = Proj(proj='utm', zone=zone_utm, ellps='WGS84')`

Note: The default geodetic system is the World Geodetic System 1984 (`WGS84`).
These `proj`, `zone`, and `ellps` details are also readable in QGIS by right-clicking on the geojson or tif file, then going to Properties/Information in the "Information from provider" section.
The Python script `Step_6_csv_from_raster_cover.py` loop over all the pixels in the .tif file, converts the position to Latitude Longitude, save the associated cover number, and writes everything to a csv file.

<img src="Illustration/Tab6.png" alt="Mon Image Locale" width="300" />

The csv file can now be used as input in the Meso-NH PRE_PGD.nam file:

`&NAM_COVER YCOVER='Portet_sur_Garonne_ECOCLIMAP.csv', YCOVERFILETYPE='ASCLLV' /`

>Notes: Be sure to specify the file extension in `YCOVER` (here, .csv) and its type in `YCOVERFILETYPE`. Do not use special characters in file names (such as '-').

## Step 7: High resolution topography

The Surfex website provides topography data at different scales and locations worldwide (http://www.umr-cnrm.fr/surfex/spip.php?article134). However, even the lowest resolution remains quite coarse for fine-mesh simulations as we intend to do. To obtain a finer topographic resolution closer to that of our ECOCLIMAP database created with Geoclimate, we can use OpenTopography https://opentopography.org/. This site gathers a number of topographic databases from around the world with resolutions of up to 1 m.

To download a geographic area of interest, go to the menu Data/DataCatalog: https://portal.opentopography.org/dataCatalog

Numerous datasets are available, and you need to find the one that suits your needs.

Note: A global database with a 30m resolution is available: https://portal.opentopography.org/raster?opentopoID=OTALOS.112016.4326.2

To download, click on SELECT A REGION in the upper right corner of the map, draw a rectangle, then deselect the GeoTiff format, fill in the Job Description zone, and you will receive the data by email.

>Note: Please make sure that the selected domain is larger than your domain in Meso-NH

<img src="Illustration/Step7.png" alt="Mon Image Locale" width="700" />

Following the same process as in Step 6, we will rasterize this pixel image into a CSV file. For this, we can use the Python script `Step_7_csv_from_raster_topo.py`. This results in a CSV file with 3 columns: `['Latitude [deg]', 'Longitude [deg]', 'Altitude [m]']`.

<img src="Illustration/Tab7.png" alt="Mon Image Locale" width="300" />

The csv file can now be used as input in the Meso-NH PRE_PGD.nam file:

`&NAM_ZS        YZS='USGS10m.csv',    YZSFILETYPE='ASCLLV' /`

Notes: Be sure to specify the file extension in `YZS` (here, .csv) and its type in `YZSFILETYPE`. Do not use special characters in file names (such as '-').


## Step 8: Plotting the Cover and Topography Map

Now that we have replaced the cover and topography databases, we can proceed with the PREP step in Meso-NH. The output is a .nc (NetCDF) file.
To visualize the covers and topography interpolated done by Meso-NH/Surfex in the PREP step, you can use the Python script `Step_8_plot_cover_orography.py`.
The legend and colors of the covers can be modified in the file `cover_color_legend.csv`, which was automatically generated in Step 3. Adjust the legend and color settings as needed to enhance the visualization of covers and topography in your output.

<img src="Illustration/Step8.png" alt="Mon Image Locale" width="700" />

<img src="Illustration/Step8.1.png" alt="Mon Image Locale" width="500" />

